In [1]:
Dir = Literal['a','b',4]


In [8]:
typing.get_origin(int)

In [4]:
typing.get_origin(Dir)

typing.Literal

In [6]:
typing.get_args(Dir)

('a', 'b', 4)

In [7]:
typing.get_args

<function typing.get_args(tp)>

In [38]:
sr=pl.datetime_range(
    dt.datetime(2024,1,1,tzinfo=UTC),
    dt.datetime(2024,1,8,23,tzinfo=UTC),
    interval='1h',
    eager=True,
).rename("datetime")

In [39]:
df=sr.to_frame().with_columns(
    x=pl.int_range(end=pl.len())
    ,
    y=pl.int_range(start=1,end=2*pl.len(), step=2),
)
df

datetime,x,y
"datetime[μs, UTC]",i64,i64
2024-01-01 00:00:00 UTC,0,1
2024-01-01 01:00:00 UTC,1,3
2024-01-01 02:00:00 UTC,2,5
2024-01-01 03:00:00 UTC,3,7
…,…,…
2024-01-08 21:00:00 UTC,189,379
2024-01-08 22:00:00 UTC,190,381
2024-01-08 23:00:00 UTC,191,383


In [55]:
from utilities.altair import *

plot=plot_intraday_dataframe(df, value_name='asf')
plot

alt.LayerChart(...)

In [49]:
df3
df4=df3.group_by('_date_index').agg(
    col('_datetime_index').min().alias('datetime index'),
    col('_datetime_index').max().alias('datetime index_max')+1,
    col('_date').first().dt.strftime("%Y-%m-%d").alias('date_label'),
).sort('_date_index')
df4

datetime,x,y,_datetime_index,_date,_date_index
"datetime[μs, UTC]",i64,i64,i64,date,i64
2024-01-01 00:00:00 UTC,0,1,0,2024-01-01,0
2024-01-01 01:00:00 UTC,1,3,1,2024-01-01,0
2024-01-01 02:00:00 UTC,2,5,2,2024-01-01,0
2024-01-01 03:00:00 UTC,3,7,3,2024-01-01,0
…,…,…,…,…,…
2024-01-08 21:00:00 UTC,189,379,189,2024-01-08,7
2024-01-08 22:00:00 UTC,190,381,190,2024-01-08,7
2024-01-08 23:00:00 UTC,191,383,191,2024-01-08,7


_date_index,datetime index,datetime index_max,date_label
i64,i64,i64,str
0,0,24,"""2024-01-01"""
1,24,48,"""2024-01-02"""
2,48,72,"""2024-01-03"""
3,72,96,"""2024-01-04"""
…,…,…,…
5,120,144,"""2024-01-06"""
6,144,168,"""2024-01-07"""
7,168,192,"""2024-01-08"""


In [43]:
span = alt.Chart(df4).mark_rect(opacity=0.1).encode(
    x=alt.X('datetime index', title=None),x2=alt.X2('datetime index_max',title=None),y=value(0),
    color=Color('_date_index', legend=None).scale(scheme='category10')
)

In [44]:
text = alt.Chart(df4).mark_text(align='left', fontSize=9).encode(
    x='datetime index', y=value(5), text='date_label'
)

In [45]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [46]:
(
    plot + text + span
).properties(width=1000).interactive(bind_y=False)

alt.LayerChart(...)